In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import load_iris

df = load_iris(return_X_y = True, as_frame = True)
df = pd.concat((df[0], df[1]), axis = 1)
df = pd.concat((df, pd.get_dummies(df['target'], prefix = 'target')), axis = 1)


In [2]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_0,target_1,target_2
0,5.1,3.5,1.4,0.2,0,True,False,False
1,4.9,3.0,1.4,0.2,0,True,False,False
2,4.7,3.2,1.3,0.2,0,True,False,False
3,4.6,3.1,1.5,0.2,0,True,False,False
4,5.0,3.6,1.4,0.2,0,True,False,False
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,False,False,True
146,6.3,2.5,5.0,1.9,2,False,False,True
147,6.5,3.0,5.2,2.0,2,False,False,True
148,6.2,3.4,5.4,2.3,2,False,False,True


In [3]:
df.drop(columns=['target'],inplace=True)

In [4]:
from sklearn.model_selection import train_test_split

X=df.drop(['target_0','target_1','target_2'],axis=1)
y=df[["target_0","target_1","target_2"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, stratify=y, random_state=42)

In [5]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output = False)
y_train = enc.fit_transform(np.array(y_train).reshape(-1,1))
y_test = enc.transform(np.array(y_test).reshape(-1,1)) 
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)

In [18]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.losses import CategoricalCrossentropy

model = tf.keras.Sequential([
    Conv2D(16, padding='same', activation='softmax',input_shape=[150,]),
    MaxPooling2D(),
    Conv2D(32, padding='same', activation='softmax'),
    MaxPooling2D(),
    Conv2D(64, padding='same', activation='softmax'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])


tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
)


model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.build()

model.summary()

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

TypeError: Conv2D.__init__() missing 1 required positional argument: 'kernel_size'